#Scrap content from wiki page

Import packages

In [0]:
#import statements
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup as bsoup
import folium
from geopy.geocoders import Nominatim
import requests

Scrap the table from the wikipedia page

In [0]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = bsoup(req, 'html.parser')
tbody = soup.find("tbody")
rows = tbody.find_all("tr")
res = []

for i ,j in enumerate(rows):
  cnt = i-1
  #print(cnt)
  #print()
  temp = j.find_all("td")
  tmp = []
  for k in temp:
    strk = str(k)
    strk = strk.replace("<td>","")
    strk = strk.replace("</td>","")
    strk = strk.replace("\n","")
    #print(strk)
    #print()
    tmp.append(strk)
  res.append(tmp)
df = pd.DataFrame(res,columns = ['PostalCode','Borough','Neighborhood'])


Remove rows that are not assigned and group by neighborhood, concatenating with a comma

In [0]:
for i, j in df.iterrows():
  temp = "Not assigned"
  if (j[1] == temp):
    df = df.drop([i])
df=df.drop([0])
df['Neighborhood']= df.groupby(['PostalCode'])['Neighborhood'].transform(lambda x: ', '.join(x))
df.drop_duplicates()

Print df shape

In [74]:
df.shape

(103, 3)

#Create dataframe with geo properties

Download csv file

In [76]:
!wget http://cocl.us/Geospatial_data

--2020-05-15 12:45:54--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 119.81.168.76, 161.202.50.39, 119.81.168.75
Connecting to cocl.us (cocl.us)|119.81.168.76|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-05-15 12:45:54--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|119.81.168.76|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-15 12:45:58--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-15 12:45:59--  https://ibm.box.com/public/st

Create dataframe for geofile

In [78]:
geo = pd.read_csv("./Geospatial_data")
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Next is just to merge the two files

In [0]:
dfres = pd.merge(df,geo,how='left',left_on='PostalCode',right_on='Postal Code')
dfres.drop("Postal Code", axis=1, inplace=True)

In [82]:
dfres.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#Exploration thingy

Get some toronto coords

In [86]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print('The coordinates of Toronto city are {}, {}.'.format(lat, lng))

The coordinates of Toronto city are 43.6534817, -79.3839347.


New dataset without non torontos

In [96]:
dft = dfres[dfres['Borough'].str.contains('Toronto')].reset_index(drop=True)
dft.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [98]:
tmap = folium.Map(location = [lat,lng],zoom_start = 12)
for lat, lng, borough, neighborhood in zip(dft['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighborhood']):
  label = '{}, {}'.format(neighborhood, borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker([lat,lng],radius = 5, popup = label, color='blue', fill = True, fill_color = 'cyan', fill_opacity = 0.5, parse_html=False).add_to(tmap)
tmap